# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [2]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

## You must have already run a hyperparameter tuning job to analyze it here.
## The Hyperparameter tuning jobs you have run are listed in the Training section on your SageMaker dashboard.
## Copy the name of a completed job you want to analyze from that list.
## For example: tuning_job_name = 'mxnet-training-201007-0054'.
tuning_job_name = 'xgboost-tuningjob-15-13-34-13'

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [20]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
# is_minimize = (tuning_job_result['TrainingJobDefinitions'][0]['TuningObjective']['Type'] != 'Maximize')
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
# objective_name = tuning_job_result['TrainingJobDefinitions'][0]['TuningObjective']['MetricName']
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

Reminder: the tuning job has not been completed.
9 training jobs have completed


In [21]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2021, 3, 15, 13, 34, 41, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.7730029821395874},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2021, 3, 15, 13, 37, 58, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:770499558699:training-job/xgboost-tuningjob-15-13-34-13-002-ebdbf08a',
 'TrainingJobName': 'xgboost-tuningjob-15-13-34-13-002-ebdbf08a',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2021, 3, 15, 13, 37, 10, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '1.0552425434028816',
                          'eta': '0.1700836533317731',
                          'max_depth': '6',
                          'min_child_weight': '2.137744573249043'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [22]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 12
{'lowest': 0.6297420263290405, 'highest': 0.7755879759788513}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
4,1.274670,0.116098,6.0,5.808136,xgboost-tuningjob-15-13-34-13-011-2f70fcd1,Completed,0.775588,2021-03-15 13:49:16+00:00,2021-03-15 13:50:28+00:00,72.0
13,1.055243,0.170084,6.0,2.137745,xgboost-tuningjob-15-13-34-13-002-ebdbf08a,Completed,0.773003,2021-03-15 13:37:10+00:00,2021-03-15 13:37:58+00:00,48.0
14,0.033928,0.366283,3.0,3.098819,xgboost-tuningjob-15-13-34-13-001-0ae4cee4,Completed,0.771966,2021-03-15 13:38:00+00:00,2021-03-15 13:38:54+00:00,54.0
8,0.778516,0.111623,2.0,4.527778,xgboost-tuningjob-15-13-34-13-007-e053cce9,Completed,0.771483,2021-03-15 13:44:56+00:00,2021-03-15 13:46:01+00:00,65.0
9,0.778516,0.111623,2.0,4.347778,xgboost-tuningjob-15-13-34-13-006-406df19c,Completed,0.771483,2021-03-15 13:41:32+00:00,2021-03-15 13:42:37+00:00,65.0
5,1.685681,0.197408,6.0,7.476234,xgboost-tuningjob-15-13-34-13-010-13a9d074,Completed,0.769723,2021-03-15 13:48:55+00:00,2021-03-15 13:50:00+00:00,65.0
3,1.304416,0.172445,10.0,5.678321,xgboost-tuningjob-15-13-34-13-012-71db4a08,Completed,0.761239,2021-03-15 13:49:19+00:00,2021-03-15 13:50:23+00:00,64.0
10,1.255832,0.623231,4.0,1.640503,xgboost-tuningjob-15-13-34-13-005-3458fee8,Completed,0.755099,2021-03-15 13:41:08+00:00,2021-03-15 13:42:09+00:00,61.0
11,1.870762,0.401399,8.0,3.699409,xgboost-tuningjob-15-13-34-13-004-18761a12,Completed,0.747071,2021-03-15 13:40:54+00:00,2021-03-15 13:42:47+00:00,113.0
7,0.475476,0.048605,1.0,2.480750,xgboost-tuningjob-15-13-34-13-008-dcd39947,Completed,0.745639,2021-03-15 13:45:01+00:00,2021-03-15 13:45:58+00:00,57.0


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [30]:
tuner.tuning_ranges.keys()

dict_keys(['max_depth', 'eta', 'min_child_weight', 'alpha'])

In [34]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [33]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))

# deploy model

In [ ]:
REGION = boto3.session.Session().region_name
sm = boto3.client('sagemaker', region_name=REGION)

In [ ]:
def create_model(bucket, source_bucket, model_name, model_path, container, role):
    source_dir = f's3://{source_bucket}/entry-point/entry_deploy.tar.gz'
    print(f'source_dir {source_dir}')

    try:
        sm.create_model(
            ModelName=model_name,
            PrimaryContainer={
                'Image': container,
                'ModelDataUrl': f'{model_path}',
                'Environment': {
                    'SAGEMAKER_SUBMIT_DIRECTORY': source_dir,
                    'SAGEMAKER_PROGRAM': 'entrypoint.py',
                    'SAGEMAKER_REGION': 'us-west_2'
                }
            },
            ExecutionRoleArn=role
        )
    except Exception as e:
        print(f'Error while creating model {model_name}: {e}')
        raise(e)

def create_endpoint_config(config_name, model_name):
    sm.create_endpoint_config(
        EndpointConfigName=config_name,
        ProductionVariants=[
            {
                'VariantName': 'prod',
                'ModelName': model_name,
                'InitialInstanceCount': 1,
                'InstanceType': 'ml.t2.medium'
            }
        ]
    )


def create_endpoint(endpoint_name, config_name):
    sm.create_endpoint(
        EndpointName=endpoint_name,
        EndpointConfigName=config_name
    )
    
def create_model_endpoint(bucket, source_dir, endpoint_name, model_path, container, role):
    model_name = f'model-{datetime.datetime.today().strftime("%Y-%m-%d-%H-%M-%S")}'
    config_name = f'config-{datetime.datetime.today().strftime("%Y-%m-%d-%H-%M-%S")}'
    create_model(bucket, source_dir, model_name, model_path, container, role)
    create_endpoint_config(config_name, model_name)
    create_endpoint(endpoint_name, config_name)